In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
print(os.getcwd())

import platform
from matplotlib import font_manager, rc
import seaborn as sns # 데이터셋을 가져오기 위해 import

if platform.system() == 'Darwin':
    rc('font', family = 'AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname = 'c:/Windows/Fonts/malgun.ttf').get_name()
    rc('font', family = font_name)
    
# 그래프에서 음수를 사용하기 위한 설정
plt.rcParams['axes.unicode_minus'] = False

C:\Users\USER\lg dx python


In [4]:
# auto-mpg 데이터를 가져와서 이상치를 제거하고 숫자 타입으로 변환

DF = pd.read_csv('./data/auto-mpg.csv', header = None)
DF.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
             'acceleration', 'model year', 'origin', 'name']
DF['horsepower'].replace('?', np.nan, inplace = True)
DF.dropna(subset = 'horsepower', inplace = True, axis = 0)
DF['horsepower'] = DF['horsepower'].astype('float')

print(DF.head())
DF.info()

    mpg  cylinders  displacement  horsepower  weight  acceleration  \
0  18.0          8         307.0       130.0  3504.0          12.0   
1  15.0          8         350.0       165.0  3693.0          11.5   
2  18.0          8         318.0       150.0  3436.0          11.0   
3  16.0          8         304.0       150.0  3433.0          12.0   
4  17.0          8         302.0       140.0  3449.0          10.5   

   model year  origin                       name  
0          70       1  chevrolet chevelle malibu  
1          70       1          buick skylark 320  
2          70       1         plymouth satellite  
3          70       1              amc rebel sst  
4          70       1                ford torino  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           392 non-null    float64
 1   cylinders     392 non-null    int64

In [8]:
# one hot encoding 적용

# horsepower 특성을 범주형으로 추가 - 3개 영역으로 구분
# 3개의 구간으로 나누고 갯수와 경계값을 리턴 받아서 저장
count, boundaries = np.histogram(DF['horsepower'], 3)

# 범주형 형태로 생성
# 사용할 데이터, 경계값, 영역 이름, lowest 포함 여부 순서
DF['hp_bin'] = pd.cut(x = DF['horsepower'], bins = boundaries,
                     labels = ['저출력', '보통', '고출력'], include_lowest = True)

print(DF[['horsepower', 'hp_bin']].tail())

     horsepower hp_bin
393        86.0    저출력
394        52.0    저출력
395        84.0    저출력
396        79.0    저출력
397        82.0    저출력


In [11]:
# 범주형에 대해 one hot 인코딩을 수행
# 값이 3종류이므로 3개의 특성이 만들어지고 그 중 하나만 1이 됨
horsepower_dummies = pd.get_dummies(DF['hp_bin'])
print(horsepower_dummies)

     저출력  보통  고출력
0      0   1    0
1      0   1    0
2      0   1    0
3      0   1    0
4      0   1    0
..   ...  ..  ...
393    1   0    0
394    1   0    0
395    1   0    0
396    1   0    0
397    1   0    0

[392 rows x 3 columns]


In [14]:
# 1개의 특성에 대해 one hot 인코딩을 처리하는 클래스
from sklearn.preprocessing import LabelBinarizer

# 범주형 데이터에 LabelBinarizer 를 적용
one_hot = LabelBinarizer()
print(one_hot.fit_transform(DF['hp_bin']))

# 이름 확인
print(one_hot.classes_) # 순서는 달라질 수 있음

[[0 1 0]
 [0 1 0]
 [0 1 0]
 ...
 [0 0 1]
 [0 0 1]
 [0 0 1]]
['고출력' '보통' '저출력']


In [17]:
# 2개 이상의 특성을 가지고 one hot 인코딩
# 2개 이상의 1이 등장할 수 있음

# 2개 이상의 특성에 적용하는 MultiLabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer

# 데이터 생성
multi_features = [('a', 'b'), ('b', 'c'), ('c', 'a'), ('e', 'c'),
                  ('c', 'd'), ('d', 'b'), ('a', 'd'), ('d', 'e')]

# LabelBinarizer 와 동일한 방식으로 적용
multi_one_hot = MultiLabelBinarizer()
print(multi_one_hot.fit_transform(multi_features))
print(multi_one_hot.classes_)


[[1 1 0 0 0]
 [0 1 1 0 0]
 [1 0 1 0 0]
 [0 0 1 0 1]
 [0 0 1 1 0]
 [0 1 0 1 0]
 [1 0 0 1 0]
 [0 0 0 1 1]]
['a' 'b' 'c' 'd' 'e']


In [19]:
from sklearn.preprocessing import LabelEncoder

# 범주형 데이터에 LabelEncoder 를 적용
# 적용 방식은 동일함
label = LabelEncoder()
print(label.fit_transform(DF['hp_bin']))

# 이름 확인
print(label.classes_) 

[1 1 1 1 1 0 0 0 0 0 0 1 1 0 2 2 2 2 2 2 2 2 2 1 2 0 0 0 0 2 2 2 2 2 2 2 2
 1 0 1 1 0 0 0 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 0 1 1 1 0 1 1 0 2 1 1 1
 1 1 2 2 2 2 2 2 2 2 0 1 1 1 1 0 1 1 1 0 0 0 2 2 2 2 2 2 1 1 0 0 2 2 2 2 2
 2 2 2 1 0 2 2 2 1 1 1 1 0 2 2 2 2 2 2 2 2 1 2 1 1 1 1 1 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 0 1 1 1 1 2 1 2 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2
 1 1 1 1 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 1 2 1 1 0 1 1 1 2 2 2 2 2 1 1 1
 1 1 2 2 2 0 0 0 1 2 2 2 2 2 2 2 2 2 1 1 2 2 2 2 2 1 1 1 2 2 2 2 2 2 2 2 1
 1 1 1 1 1 2 2 2 2 2 2 2 2 2 1 1 1 2 2 1 2 2 2 1 1 1 1 1 1 1 1 1 2 2 2 2 2
 1 2 2 2 2 2 2 2 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2
 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 1 2 2 1 2 2 2 2 2 2 2 2]
['고출력' '보통' '저출력']


In [21]:
# 순서가 의미를 갖는 경우 - replace 활용하기

# 데이터 생성
df = pd.DataFrame({'score' : ['수', '우', '미', '수', '양', '가', '미', '우']})
# 순서를 지정하기 위해 dict 생성
scale_mapper = {'가' : 1, '양' : 2, '미' : 3, '우' : 4, '수' : 5}

df['encoder'] = df['score'].replace(scale_mapper)
print(df)

  score  encoder
0     수        5
1     우        4
2     미        3
3     수        5
4     양        2
5     가        1
6     미        3
7     우        4


In [24]:
from sklearn.preprocessing import OrdinalEncoder

# 데이터 생성
features = np.array([['low' , 10], ['middle' , 20], ['high' , 14]])

# 객체 생성
ordinal = OrdinalEncoder()
# 생성한 객체에 데이터 삽입
print(ordinal.fit_transform(features))

print(ordinal.categories_) # 사전 순서대로 순서를 지정함

[[1. 0.]
 [2. 2.]
 [0. 1.]]
[array(['high', 'low', 'middle'], dtype='<U11'), array(['10', '14', '20'], dtype='<U11')]


In [34]:
# 분류 모델을 이용한 결측값 대체
from sklearn.neighbors import KNeighborsClassifier

# 훈련 데이터 생성
data = np.array([[0, 2, 1.3], [4, 2, 0.2], [0, 2 , 4.1], [2, 3, 2.5]])

# 예측에 사용할 데이터
data_with_na = np.array([[np.nan, 0.2, 4], [np.nan, 3.1, 1.6]])

# KNN 학습기 생성 - 거리를 사용
clf = KNeighborsClassifier(3, weights = 'distance')

# 첫번째 데이터를 label로 하고 나머지 데이터를 feature로 설정해서 훈련 진행
trained_model = clf.fit(data[:, 1:], data[:, 0])

# 생성한 모델과 na가 포함된 데이터의 다른 데이터들을 통해 예측 - predict
imputed_values = trained_model.predict(data_with_na[:, 1:])

# 결과 출력
print(imputed_values)

# 예측한 데이터와 원본 데이터를 합치기 - 데이터의 na 를 예측값으로 대체
# hstack 은 옆으로 데이터를 합침 - 괄호 2개 필요
data_with_imputed = np.hstack((imputed_values.reshape(-1, 1), data_with_na[:, 1:]))
print(data_with_imputed)

# 결측치를 대체한 데이터와 훈련에 사용한 데이터를 합침
# vstack은 데이터를 위아래로 합침 - 괄호 2개 필요
result = np.vstack((data_with_imputed, data))
print(result)

[0. 2.]
[[0.  0.2 4. ]
 [2.  3.1 1.6]]
[[0.  0.2 4. ]
 [2.  3.1 1.6]
 [0.  2.  1.3]
 [4.  2.  0.2]
 [0.  2.  4.1]
 [2.  3.  2.5]]


In [37]:
# 가장 많이 나오는 데이터로 대체

from sklearn.impute import SimpleImputer

# nan 이 포함된 데이터를 기존 데이터와 합침
data_complete = np.vstack((data_with_na, data))
print(data_complete)

# 최빈값으로 결측치를 대체
imputer = SimpleImputer(strategy = 'most_frequent')
# 데이터에 적용
imputer.fit_transform(data_complete)

[[nan 0.2 4. ]
 [nan 3.1 1.6]
 [0.  2.  1.3]
 [4.  2.  0.2]
 [0.  2.  4.1]
 [2.  3.  2.5]]


array([[0. , 0.2, 4. ],
       [0. , 3.1, 1.6],
       [0. , 2. , 1.3],
       [4. , 2. , 0.2],
       [0. , 2. , 4.1],
       [2. , 3. , 2.5]])

## 정규 표현식

In [48]:
import re

# 매칭 여부를 확인
# 패턴에 일치하는 데이터가 있으면 Match 객체를 리턴하고 없으면 None 리턴
# 앞의 표현식과 뒤의 데이터를 비교
match = re.match('[0-9]', '1234')
print(match) # Match object 출력
match = re.match('[0-9]', 'jongho')
print(match) # None 출력
match = re.match('[0-9]', 'jongho33')
print(match) # None
match = re.match('[0-9]', ' 1234 ')
print(match) # 공백이 포함되어 있으므로 일치하지 않아서 None
match = re.match('^\s[0-9]', ' 1234')
print(match) # 패턴에 공백이 포함되어서 Match object

<re.Match object; span=(0, 1), match='1'>
None
None
None
<re.Match object; span=(0, 2), match=' 1'>


In [50]:
string = '!@#as fQ R81  #.가나다라 '

# 숫자 데이터 제거

# 숫자 패턴을 입력(compile)
p_num = re.compile('[0-9]+')
# 숫자 패턴과 일치하는 것을 ''(제거)로 대체
result = p_num.sub('', string)
print(result) # '!@#as fQ R  #.가나다라 ' - 숫자 사라짐

!@#as fQ R  #.가나다라 


In [51]:
# 특수문자 제거

# \W 는 숫자나 문자를 제외한 전부
p_spe = re.compile('\W+')
result = p_spe.sub('', result)
# 공백도 사라짐
print(result) # 'asfQR가나다라'

asfQR가나다라


In [2]:
# unicodedata 사용

import unicodedata
import sys

string_data = ['이렇게.', '저렇게!', '이런,', '저런"']

# 구두점에 대한 dict 생성
punc = dict.fromkeys(i for i in range(sys.maxunicode)
                     if unicodedata.category(chr(i)).startswith('P'))

# 구두점을 포함해서 특수 문자들을 삭제
result = [string.translate(punc) for string in string_data]
print(result)

['이렇게', '저렇게', '이런', '저런']


In [57]:
# nltk 설치

!pip install nltk

In [58]:
# nltk 패키지 설치
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [11]:
from nltk.tokenize import word_tokenize # 단어 단위로 토큰화
from nltk.tokenize import sent_tokenize # 문장 단위로 토큰화

sentence = 'a b c d erdfas 빌뮇바누히'

# 공백 단위로 구분해서 토큰화
tok = word_tokenize(sentence)
print(tok)
# ['a', 'b', 'c', 'd', 'erdfas', '빌뮇바누히']

# 구두점과 띄어쓰기를 기준으로 문장을 구분해서 토큰화
sentences = '. 1번 문장입니다. 2번 문장. 3번 문장, ㅁㄴㅇㄹ.'
print(sent_tokenize(sentences)) 
# ['.', '1번 문장입니다.', '2번 문장.', '3번 문장, ㅁㄴㅇㄹ.']

['a', 'b', 'c', 'd', 'erdfas', '빌뮇바누히']
['.', '1번 문장입니다.', '2번 문장.', '3번 문장, ㅁㄴㅇㄹ.']


## 불용어 처리

In [8]:
# 불용어 제거 - 한글은 불용어 사전이 따로 없기 때문에 직접 만들어야 함
word_korean = ['1월', '3일', '2023년', '10시', '08분', '25초']
korean_stopwords = ['1월', '25초']

# 'i for i ~ ' 는 작업을 수행해서 generator 를 생성
# generator 는 이터레이터로 접근할 수 있는 객체임
print([i for i in word_korean if i not in korean_stopwords])

['3일', '2023년', '10시', '08분']


In [6]:
# 영문은 nltk 에서 기본적인 불용어 사전을 제공함

from nltk.corpus import stopwords

word_eng = ['and', 'the good', 'nice', 'c', 'take', 'movie', 'hands', 'a']

# 영문 불용어 사전에 포함되지 않는 단어만 가져오도록
result = [word for word in word_eng if word not in stopwords.words('english')]

print(result) # and, a 를 제외함

['the good', 'nice', 'c', 'take', 'movie', 'hands']


In [10]:
# sklearn 의 불용어의 수가 nltk 보다 더 많음
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

result = [word for word in word_eng if word not in ENGLISH_STOP_WORDS]
print(result) # 위에 더해서 'take' 도 제외됨

['the good', 'nice', 'c', 'movie', 'hands']


In [20]:
# 영문의 어간 추출
from nltk.stem import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer

string = "All Pythons have python's python."

# 단어 토큰화 - word tokenize, 공백을 기준으로 분할하여 list 생성
result1 = word_tokenize(string)
print(result1) # python 과 's, . 을 분리하여 토큰화

# 어간 추출 - PerterStemmer
ps_stemmer = PorterStemmer()
for word in result1:
    print(ps_stemmer.stem(word), end = '\t')
# 대문자와 소문자를 구별하지 않으며 복수형과 단수형도 상관없이 추출

print()

# 어간 추출 - LancasterStemmer
ps_stemmer = LancasterStemmer()
for word in result1:
    print(ps_stemmer.stem(word), end = '\t')

['All', 'Pythons', 'have', 'python', "'s", 'python', '.']
all	python	have	python	's	python	.	
al	python	hav	python	's	python	.	

In [21]:
# 형태소 분석 - 품사 태깅
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [22]:
from nltk import pos_tag, word_tokenize

tokens = word_tokenize('The movie was made by that movie star!')
# 토큰을 가지고 단어와 품사의 튜플 리스트로 나눔
en_tags = pos_tag(tokens)
# 품사 : NNP - 고유명사, NN : 명사, RB : 부사, VBD : 동사 ...

print(en_tags) # [('The', 'DT'), ('movie', 'NN'), ('was', 'VBD'), ... ('!', '.')]

[('The', 'DT'), ('movie', 'NN'), ('was', 'VBD'), ('made', 'VBN'), ('by', 'IN'), ('that', 'DT'), ('movie', 'NN'), ('star', 'NN'), ('!', '.')]


In [23]:
# 품사가 명사(NN)인 단어만 출력
# tuple 이기 때문에 나눠서 받아올 수 있음
print([word for word, tag in en_tags if tag == 'NN'])

['movie', 'movie', 'star']


In [25]:
!pip install konlpy

  Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)


In [27]:
conda install -c conda-forge jpype1

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.




==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.7.2

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.7.2




In [3]:
from konlpy.tag import Kkma

In [15]:
text = '''오늘도 날씨가 매우 덥다. 언제쯤 시원해지려나. 다음주에 비가 오면 좋겠다.
그 다음 문장입니다. 그 다다음 문장.'''

kkma = Kkma()

# 문장 분석
print(kkma.sentences(text))

# 단어 분석
print(kkma.nouns(text))

# 형태소 분석
print(kkma.pos(text))

['오늘도 날씨가 매우 덥다.', '언제쯤 시원 해지려나.', '다음주에 비가 오면 좋겠다.', '그 다음 문장입니다.', '그 다다음 문장.']
['오늘', '날씨', '언제쯤', '다음주', '비가', '다음', '문장', '다다']
[('오늘', 'NNG'), ('도', 'JX'), ('날씨', 'NNG'), ('가', 'JKS'), ('매우', 'MAG'), ('덥', 'VA'), ('다', 'EFN'), ('.', 'SF'), ('언제쯤', 'NNG'), ('시원', 'XR'), ('해지', 'VV'), ('려나', 'EFQ'), ('.', 'SF'), ('다음주', 'NNG'), ('에', 'JKM'), ('비가', 'NNG'), ('오', 'VV'), ('면', 'ECE'), ('좋', 'VA'), ('겠', 'EPT'), ('다', 'EFN'), ('.', 'SF'), ('그', 'MDT'), ('다음', 'NNG'), ('문장', 'NNG'), ('이', 'VCP'), ('ㅂ니다', 'EFN'), ('.', 'SF'), ('그', 'MDT'), ('다다', 'NNP'), ('음', 'XSN'), ('문장', 'NNG'), ('.', 'SF')]


In [16]:
# 다른 형태소 분석기인 Hannanum
# 성능은 Kkma가 우수하다고 알려져 있지만
# 메모리 사용량이 많고 속도가 조금 느리다는게 단점

from konlpy.tag import Hannanum

text = '''오늘도 날씨가 매우 덥다. 언제쯤 시원해지려나. 다음주에 비가 오면 좋겠다.
그 다음 문장입니다. 그 다다음 문장.'''
han = Hannanum()


# 단어 분석
print(han.nouns(text))
# 형태소 분석
print(han.morphs(text))
print(han.pos(text))

['오늘', '날씨', '언제쯤', '시원', '다음주', '비', '다음', '문장', '다다음', '문장']
['오늘', '도', '날씨', '가', '매우', '덥', '다', '.', '언제쯤', '시원', '하', '어', '지', '려', '나', '.', '다음주', '에', '비', '가', '오', '면', '좋', '겠다', '.', '그', '다음', '문장', '이', 'ㅂ니다', '.', '그', '다다음', '문장', '.']
[('오늘', 'N'), ('도', 'J'), ('날씨', 'N'), ('가', 'J'), ('매우', 'M'), ('덥', 'P'), ('다', 'E'), ('.', 'S'), ('언제쯤', 'N'), ('시원', 'N'), ('하', 'X'), ('어', 'E'), ('지', 'P'), ('려', 'E'), ('나', 'J'), ('.', 'S'), ('다음주', 'N'), ('에', 'J'), ('비', 'N'), ('가', 'J'), ('오', 'P'), ('면', 'E'), ('좋', 'P'), ('겠다', 'E'), ('.', 'S'), ('그', 'M'), ('다음', 'N'), ('문장', 'N'), ('이', 'J'), ('ㅂ니다', 'E'), ('.', 'S'), ('그', 'M'), ('다다음', 'N'), ('문장', 'N'), ('.', 'S')]


In [21]:
# BoW(Bag of Word - 단어의 갯수)

from sklearn.feature_extraction.text import CountVectorizer

text_data = np.array(['I got a day after import feature', 'I like to like I',
                     'My phone is light like a day', 'Dear my friend'])

# 객체 생성
cnt = CountVectorizer()
# 데이터 삽입
bag_of_words = cnt.fit_transform(text_data)

# 희소 행렬의 형태로 결과가 출력됨
#print(bag_of_words) 
# 밀집 행렬의 형태로 전환해서 출력
print(bag_of_words.toarray())
# 특성의 이름 확인 - 자동으로 정렬을 진행하며 stopwords 도 처리
print(cnt.get_feature_names_out()) # I, a 등의 stopwords 가 없음

[[1 1 0 1 0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 2 0 0 1]
 [0 1 0 0 0 0 0 1 1 1 1 1 0]
 [0 0 1 0 1 0 0 0 0 0 1 0 0]]
['after' 'day' 'dear' 'feature' 'friend' 'got' 'import' 'is' 'light'
 'like' 'my' 'phone' 'to']


In [26]:
# tf-idf

from sklearn.feature_extraction.text import TfidfVectorizer

text_data = np.array(['I got a day after', 'I like to like a day',
                     'My phone is light like a day', 'A day likes my phones',
                     'Day after my phone'])

# tf-idf 객체를 생성
tf_idf = TfidfVectorizer()
# 데이터 삽입
result = tf_idf.fit_transform(text_data)

# 결과 출력
#print(result)
print(tf_idf.vocabulary_)
print(result.toarray())
print(tf_idf.get_feature_names_out()) 

{'got': 2, 'day': 1, 'after': 0, 'like': 5, 'to': 10, 'my': 7, 'phone': 8, 'is': 3, 'light': 4, 'likes': 6, 'phones': 9}
[[0.58873218 0.34771471 0.72971837 0.         0.         0.
  0.         0.         0.         0.         0.        ]
 [0.         0.24345993 0.         0.         0.         0.82442698
  0.         0.         0.         0.         0.510928  ]
 [0.         0.23892851 0.         0.50141831 0.50141831 0.40454114
  0.         0.33580569 0.40454114 0.         0.        ]
 [0.         0.29131278 0.         0.         0.         0.
  0.61135259 0.40942995 0.         0.61135259 0.        ]
 [0.57373967 0.33885989 0.         0.         0.         0.
  0.         0.47625576 0.57373967 0.         0.        ]]
['after' 'day' 'got' 'is' 'light' 'like' 'likes' 'my' 'phone' 'phones'
 'to']
